In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import time



In [49]:
# device confic GPU kullanmak için CPU yerine
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)


cuda:0


In [ ]:
# prepocess

def read_image(path,num_img):
  array = np.zeros([num_img,64*32])
  i= 0
  for img in os.listdir(path):
    img_path = path + "/" + img
    img = Image.open(img_path,mode = "r")
    data= np.asarray(img, dtype = "uint8")
    data = data.flatten()
    array[i,:]= data
    i+=1
    print(i)
  return array

# --------------------------------------------------------------
# READ TRAİN NEGATİVE
train_negative_path =r"/content/drive/MyDrive/Colab Notebooks/Advance_Deep_Learning/data/LSIFIR/Classification/Train/neg"
num_train_negative_img = 43390

train_negative_array = read_image(train_negative_path,num_train_negative_img)
x_train_negative_tensor = torch.from_numpy(train_negative_array)
y_train_negative_tensor = torch.zeros(num_train_negative_img,dtype=torch.long)

# READ TRAİN PoZİTİVE
train_positive_path = "/content/drive/MyDrive/Colab Notebooks/Advance_Deep_Learning/data/LSIFIR/Classification/Train/pos"
num_train_pozitive_img = 10208
train_pozitive_array = read_image(train_positive_path,num_train_pozitive_img)
x_train_pozitive_tensor = torch.from_numpy(train_pozitive_array)
y_train_pozitive_tensor = torch.ones(num_train_pozitive_img,dtype=torch.long)


# negatif ve pozitif resimleri train datasetini birlşetirme

x_train = torch.cat((x_train_negative_tensor,x_train_pozitive_tensor),0)
y_train = torch.cat((y_train_negative_tensor,y_train_pozitive_tensor),0)
print(x_train.size())
print(y_train.size())

# --------------------------------------------------------------


In [ ]:
# READ TEST NEGATİVE
test_negative_path ="/content/drive/MyDrive/Colab Notebooks/Advance_Deep_Learning/data/LSIFIR/Classification/Test/neg" 
num_test_negative_img = 22050

test_negative_array = read_image(test_negative_path,num_test_negative_img)
x_test_negative_tensor = torch.from_numpy(test_negative_array[:20855,:])    # 20855 başarıyı etkileyen bir durumdan doaçyı kısıtlandı
y_test_negative_tensor = torch.zeros(20855,dtype=torch.long)

# READ TEST PoZİTİVE
test_positive_path = "/content/drive/MyDrive/Colab Notebooks/Advance_Deep_Learning/data/LSIFIR/Classification/Test/pos"

num_test_pozitive_img = 5944
test_pozitive_array = read_image(test_positive_path,num_test_pozitive_img)
x_test_pozitive_tensor = torch.from_numpy(test_pozitive_array)
y_test_pozitive_tensor = torch.ones(num_test_pozitive_img,dtype=torch.long)


# negatif ve pozitif resimleri train datasetini birlşetirme

x_test = torch.cat((x_test_negative_tensor,x_test_pozitive_tensor),0)
y_test = torch.cat((y_test_negative_tensor,y_test_pozitive_tensor),0)
print(x_test.size())
print(y_test.size())

In [ ]:
# DATA GÖRSELŞTİRME

plt.imshow(x_train[36666,:].reshape(64,32),cmap="gray")


In [58]:
# CNN yapısı oluşturma
# hyper_paremater
num_epochs = 1
num_classes = 2
batch_size = 3000  # her bir iterasyonda kaç resim kullansın
learning_rate = 0.00001
# ------------------
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()

    self.conv1 = nn.Conv2d(1,10,5) # 1=> input ,10=> output channel ,5=> kernel size(5,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(10,16,5)
    self.fc1 = nn.Linear(16*13*5,520) # full connect layer buraya çalış
    self.fc2 = nn.Linear(520,130) # full connect layer buraya çalış
    self.fc3 = nn.Linear(130,num_classes) # full connect layer buraya çalış

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,16*13*5)  # Flatten yaptığımız yer
    x =F.relu(self.fc1(x)) 
    x =F.relu(self.fc2(x)) 
    x =self.fc3(x)
    return x 
   

In [59]:
train =  torch.utils.data.TensorDataset(x_train,y_train)
trainLoader = torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True)

test =  torch.utils.data.TensorDataset(x_test,y_test)
testLoader = torch.utils.data.DataLoader(test,batch_size=batch_size,shuffle=False)

In [60]:
net = Net()

In [61]:
#Loss and Optimizer

Criterion = nn.CrossEntropyLoss()
import torch.optim as optim
# optimizer = optim.SGD(net.parameters(),lr=self.learning_rate,momentum =0.8)
optimizer = optim.SGD(net.parameters(),learning_rate,momentum=0.8)

In [ ]:
#TRAİN

start = time.time()
train_acc = []
test_acc = []
lost_list = []
use_gpu = False

for epoch in range(num_epochs):
  for i,data in enumerate(trainLoader,0):
    inputs,labels = data
    inputs = inputs.view(batch_size,1,64,32) #reshape

    print("d")
    inputs = inputs.float()
    
    #use gpu
    if use_gpu:
      if torch.cuda.is_available():
        inputs ,labels = inputs.to(device),labels.to(device)

    # zero Gradient

    optimizer.zero_grad()

    #forward

    outputs = net(inputs)

    #loss
    loss= Criterion(outputs,labels)

    #back
    loss.backward()

    #update weig

    optimizer.step()
  

  # test
  total =0
  correct= 0
  with torch.no_grad():
      for data in testLoader:
        images, labels = data

        images = images.view(batch_size,1,64,32)
        images = images.float()
        
        # use gpu
        if use_gpu:
          if torch.cuda.is_available():
            images ,labels = images.to(device),labels.to(device)

      
      outputs = net(images)
      _,predicted = torch.max(outputs.data,1)
      total +=labels.size(0)
      correct += (predicted == labels).sum().items()
  acc1 = 100*correct/total
  print("accurancy test:",acc1)
  test_acc.append(acc1)



    # test
  total =0
  correct= 0
  with torch.no_grad():
      for data in trainLoader:
        images, labels = data

        images = images.view(batch_size,1,64,32)
        images = images.float()
        
        # use gpu
        if use_gpu:
          if torch.cuda.is_available():
            images ,labels = images.to(device),labels.to(device)

      
      outputs = net(images)
      _,predicted = torch.max(outputs.data,1)
      total +=labels.size(0)
      correct += (predicted == labels).sum().items()
  acc2 = 100*correct/total
  print("accurancy train:",acc2)
  train_acc.append(acc2)





print("train is done")


end = time.time()


processTime = (end-start)/60

print("Process Time:",time.process_time)